# Matrix Multiplication with Blocking and Tiling

In this notebook, we will implement **matrix multiplication** with **blocking** and **tiling** strategies in Triton to optimize performance. Blocking and tiling improve memory locality and computational efficiency, which are essential for high-performance GPU operations, especially on large matrices.

## Introduction

When working with large matrices, memory and compute requirements can be significant. Blocking and tiling can help optimize these operations by improving data locality and parallelism, maximizing the effective use of GPU resources.

### Key Terms
- **Blocking**: A technique that splits large data into smaller blocks to fit into faster, smaller memory like shared memory on a GPU. Each block is processed independently, which can lead to better cache utilization and reduced memory access latency.
  
- **Tiling**: A strategy where the computation is split into tiles that fit into the GPU’s shared memory. Tiling allows the GPU to load each tile once into shared memory and reuse it for multiple computations, reducing memory bandwidth requirements and improving performance.

These methods help reduce the number of memory accesses and allow the GPU to work on small, manageable pieces of data, which is critical for efficiency in large-scale computations.

---

## Objectives
1. **Implement matrix multiplication** using blocking and tiling in Triton.
2. **Benchmark performance** against PyTorch’s CUDA matrix multiplication.
3. **Analyze the impact of blocking and tiling** on GPU memory utilization and performance.

---

### Setting Up the Triton Kernel for Matrix Multiplication

We start by defining a Triton kernel for matrix multiplication. This kernel will load blocks of the input matrices into shared memory, perform the multiplication for each block, and accumulate the results in a final matrix. 


In [1]:
import torch
import triton
import triton.language as tl

# Triton kernel for matrix multiplication with blocking and tiling
@triton.jit
def matmul_kernel(a_ptr, b_ptr, c_ptr, M, N, K, BLOCK_SIZE: tl.constexpr):
    # Program ID in the 2D grid
    pid_m = tl.program_id(0)
    pid_n = tl.program_id(1)

    # Define tile/block start points in matrix A and B
    offsets_m = pid_m * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    offsets_n = pid_n * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    
    # Initialize accumulator for the output block
    acc = tl.zeros((BLOCK_SIZE, BLOCK_SIZE), dtype=tl.float32)
    
    # Loop over tiles in the shared K dimension
    for k in range(0, K, BLOCK_SIZE):
        # Load tiles of A and B
        a_tile = tl.load(a_ptr + offsets_m[:, None] * K + (k + tl.arange(0, BLOCK_SIZE)[None, :]), mask=(offsets_m[:, None] < M) & (k + tl.arange(0, BLOCK_SIZE)[None, :] < K), other=0.0)
        b_tile = tl.load(b_ptr + (k + tl.arange(0, BLOCK_SIZE)[:, None]) * N + offsets_n[None, :], mask=(k + tl.arange(0, BLOCK_SIZE)[:, None] < K) & (offsets_n[None, :] < N), other=0.0)
        
        # Perform the matrix multiplication for the tile
        acc += tl.dot(a_tile, b_tile)
    
    # Write the computed block to the output matrix
    tl.store(c_ptr + offsets_m[:, None] * N + offsets_n[None, :], acc, mask=(offsets_m[:, None] < M) & (offsets_n[None, :] < N))


ModuleNotFoundError: No module named 'triton'

### Explanation of the Code
- **Kernel Definition**: The `matmul_kernel` function is decorated with `@triton.jit` for Triton’s JIT compilation.
- **Grid Setup**: Each GPU thread block computes one tile of the result matrix `C`.
- **Tiling and Blocking**:
  - `offsets_m` and `offsets_n` define the starting positions of each tile within matrices `A` and `B`.
  - Each tile is loaded into shared memory using the `tl.load` operation, enabling reuse within the tile computation.
- **Loop Over K Dimension**: The loop iterates over the shared dimension `K`, loading one tile at a time and accumulating the results in `acc`.
- **Result Storage**: The result tile is stored back into `c_ptr`, which represents the output matrix.

---

## Running the Matrix Multiplication with Tiling

Let's define a wrapper function to set up the input matrices and launch the Triton kernel.


In [2]:
def matmul_tiled(a: torch.Tensor, b: torch.Tensor, BLOCK_SIZE=128):
    # Matrix dimensions
    M, K = a.shape
    _, N = b.shape
    c = torch.empty((M, N), device='cuda', dtype=torch.float32)
    
    # Define grid size for kernel launch
    grid = (M // BLOCK_SIZE, N // BLOCK_SIZE)
    
    # Launch the Triton kernel
    matmul_kernel[grid](a, b, c, M, N, K, BLOCK_SIZE=BLOCK_SIZE)
    return c

---

### Benchmarking the Tiled Matrix Multiplication

Now that we have the tiled matrix multiplication kernel, let's benchmark it against PyTorch's CUDA matrix multiplication (`torch.mm`) to see how our blocking and tiling implementation performs.

We’ll vary the block size to understand its impact on throughput and identify the optimal block size for our setup.


In [3]:
import time
import matplotlib.pyplot as plt

# Benchmark function for Triton vs PyTorch CUDA
def benchmark_matmul(M, N, K, block_sizes, repetitions=10):
    # Initialize matrices
    a = torch.rand((M, K), device='cuda', dtype=torch.float32)
    b = torch.rand((K, N), device='cuda', dtype=torch.float32)
    results = {}

    # Triton matrix multiplication with varying block sizes
    for block_size in block_sizes:
        triton_times = []
        for _ in range(repetitions):
            start = time.time()
            matmul_tiled(a, b, BLOCK_SIZE=block_size)
            torch.cuda.synchronize()
            triton_times.append(time.time() - start)
        
        avg_time = sum(triton_times) / repetitions
        gbps = 2 * M * N * K * a.element_size() * 1e-9 / avg_time
        results[f'Triton (BLOCK_SIZE={block_size})'] = (avg_time, gbps)

    # PyTorch CUDA matrix multiplication
    cuda_times = []
    for _ in range(repetitions):
        start = time.time()
        torch.mm(a, b)
        torch.cuda.synchronize()
        cuda_times.append(time.time() - start)
    
    avg_time = sum(cuda_times) / repetitions
    gbps = 2 * M * N * K * a.element_size() * 1e-9 / avg_time
    results['CUDA (Torch)'] = (avg_time, gbps)

    return results

# Define matrix dimensions and block sizes for testing
M, N, K = 1024, 1024, 1024
block_sizes = [64, 128, 256]
benchmark_results = benchmark_matmul(M, N, K, block_sizes)

# Print benchmark results
print(f"{'Configuration':<25} {'Avg Time (s)':<15} {'Throughput (GB/s)':<20}")
for config, (avg_time, gbps) in benchmark_results.items():
    print(f"{config:<25} {avg_time:<15.5f} {gbps:<20.2f}")


AssertionError: Torch not compiled with CUDA enabled

### Plotting the Results

We will visualize the throughput (GB/s) for each block size to analyze the effect of tiling and blocking.


In [4]:
# Plot the throughput results
configurations = list(benchmark_results.keys())
throughput_values = [benchmark_results[config][1] for config in configurations]

plt.figure(figsize=(10, 6))
plt.bar(configurations, throughput_values, color=['teal'] * len(block_sizes) + ['darkorange'], width=0.5)
plt.xlabel("Configuration", fontsize=14)
plt.ylabel("Throughput (GB/s)", fontsize=14)
plt.title("Matrix Multiplication Throughput: Triton Block Sizes vs CUDA (Torch)", fontsize=16)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)

# Annotate throughput values on the bars
for i, v in enumerate(throughput_values):
    plt.text(i, v + 0.5, f"{v:.2f} GB/s", ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

NameError: name 'benchmark_results' is not defined